In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip "/content/drive/My Drive/facial-expression-dataset.zip" -d "/content/drive/My Drive/facial-expression-dataset/"

In [ ]:
import numpy as np
import os

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.3.0


In [ ]:
for expression in os.listdir('/content/drive/My Drive/facial-expression-dataset/facial-expression-dataset/'):
    print(str(len(os.listdir('/content/drive/My Drive/facial-expression-dataset/facial-expression-dataset/'+expression))) + " " + expression + " images")

5121 fear images
8988 happy images
6198 neutral images
6077 sad images
4002 surprise images
4953 angry images


In [ ]:
img_size = 48
batch_size=16

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_gen = train_datagen.flow_from_directory(
    "/content/drive/My Drive/facial-expression-dataset/facial-expression-dataset/",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode = 'grayscale',
    subset='training') # set as training data

val_gen = train_datagen.flow_from_directory(
    "/content/drive/My Drive/facial-expression-dataset/facial-expression-dataset/", # same directory as training data
    target_size=(img_size, img_size),
    batch_size=batch_size,
    color_mode = 'grayscale',
    subset='validation')

Found 28274 images belonging to 6 classes.
Found 7065 images belonging to 6 classes.


In [ ]:
from tensorflow.keras import callbacks

In [ ]:
class CustomSaver(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch == 19 or epoch==39 or epoch==59 or epoch==79 or epoch==99:  # or save after some epoch, each k-th epoch etc.
            count = epoch+1
            self.model.save("/content/drive/My Drive/facial_expression_model/model_epoch_{}_loss_{}.h5".format(count, logs["loss"]))
            print("\nEpoch {} over. Saving model".format(count))

model_saver = CustomSaver()

In [ ]:
##create model
#conv2d layer 1
img_input = Input(shape=(48,48,1))
X = Conv2D(filters=64, kernel_size=(3,3), padding='same')(img_input)
print(X.shape)
X = BatchNormalization()(X)
print(X.shape)
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(2,2))(X)
print(X.shape)
X = Dropout(rate=0.25)(X)

(None, 48, 48, 64)
(None, 48, 48, 64)
(None, 24, 24, 64)


In [ ]:
#conv2d layer 2
X = Conv2D(filters=128, kernel_size=(5,5), padding='same')(X)
print(X.shape)
X = BatchNormalization()(X)
print(X.shape)
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(2,2))(X)
print(X.shape)
X = Dropout(rate=0.2)(X)

(None, 24, 24, 128)
(None, 24, 24, 128)
(None, 12, 12, 128)


In [ ]:
#conv2d layer 3
X = Conv2D(filters=512, kernel_size=(3,3), padding='same')(X)
print(X.shape)
X = BatchNormalization()(X)
print(X.shape)
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(2,2))(X)
print(X.shape)
X = Dropout(rate=0.2)(X)

#conv2d layer 4
X = Conv2D(filters=512, kernel_size=(3,3), padding='same')(X)
print(X.shape)
X = BatchNormalization()(X)
print(X.shape)
X = Activation('relu')(X)
X = MaxPooling2D(pool_size=(2,2))(X)
print(X.shape)
X = Dropout(rate=0.25)(X)

(None, 12, 12, 512)
(None, 12, 12, 512)
(None, 6, 6, 512)
(None, 6, 6, 512)
(None, 6, 6, 512)
(None, 3, 3, 512)


In [ ]:
#flatten
X = Flatten()(X)
print(X.shape)
#Dense layer 1
X = Dense(256)(X)
print(X.shape)
X = BatchNormalization()(X)
X = Activation('relu')(X)
X = Dropout(rate=0.2)(X)

#Dense layer 2
X = Dense(512)(X)
print(X.shape)
X = BatchNormalization()(X)
X = Activation('relu')(X)
X = Dropout(rate=0.2)(X)

output = Dense(6, activation='softmax')(X)

model = Model(img_input, output)
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'],
)

(None, 4608)
(None, 256)
(None, 512)


In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)      

In [ ]:
r = model.fit(
  train_gen,
  validation_data=val_gen,
  epochs=100,
  callbacks = [model_saver]
)

Epoch 1/100
1022/1768 [================>.............] - ETA: 1:13:08 - loss: 1.7377 - accuracy: 0.3122